<a href="https://colab.research.google.com/github/aaalexlit/hacking-human-vasculature/blob/main/notebooks/baseline_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!git clone https://github.com/aaalexlit/hacking-human-vasculature.git
!pip install ultralytics
!pip install wandb

In [2]:
import os
from pathlib import Path
cur_path = Path.cwd()
os.environ['YOLO_CONFIG_DIR'] = str(cur_path)

In [3]:
%%writefile /content/settings.yaml
settings_version: 0.0.4
datasets_dir: /content
weights_dir: weights
runs_dir: runs
uuid: 569f3ba64b326db489132663f79cd37279811de477381b83ac131e6cdd129cbb
sync: true
api_key: ''
openai_api_key: ''
clearml: true
comet: true
dvc: true
hub: true
mlflow: true
neptune: true
raytune: true
tensorboard: true
wandb: true


Writing /content/settings.yaml


In [4]:
import yaml
train_path = 'train'
val_path = 'val'
test_path = 'test'
dataset_path = './hacking-human-vasculature/dataset'
names = {0: 'Vessel'}
yaml_path = 'yolo_dataset.yaml'

# Create a dictionary with the required content
yaml_data = {
    'names': names,
    'nc': len(names),
    'test': test_path,
    'train': train_path,
    'val': val_path,
    'path': dataset_path
}

# Write the dictionary to a YAML file
with open(yaml_path, 'w') as file:
    yaml.dump(yaml_data, file, default_flow_style=False)

In [5]:
import wandb
import getpass

**Important note:**.
Uncomment and run the following cell if you want to disable logging to WandB  
And don't run the one after next.

In [6]:
# os.environ["WANDB_DISABLED"] = "true"

In [7]:
wandb.login(key=getpass.getpass())

··········


wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [8]:
from ultralytics import YOLO

project = 'blood_vessel_segmentation'


Train image size 1024 batching 4 images

In [9]:
model = YOLO('yolov8n-seg.pt')

name = 'train_20_epochs_1024_batch_4'

results = model.train(data=yaml_path,
                      project=project,
                      name=name,
                      epochs=20,
                      patience=5,
                      batch=4,
                      imgsz=1024,
                      cache=True
                     )

100%|██████████| 6.73M/6.73M [00:00<00:00, 79.4MB/s]


Ultralytics YOLOv8.1.3 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=segment, mode=train, model=yolov8n-seg.pt, data=yolo_dataset.yaml, epochs=20, time=None, patience=5, batch=4, imgsz=1024, save=True, save_period=-1, cache=True, device=None, workers=8, project=blood_vessel_segmentation, name=train_20_epochs_1024_batch_4, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, sho

100%|██████████| 755k/755k [00:00<00:00, 16.3MB/s]


Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

wandb: Currently logged in as: aaalex-lit. Use `wandb login --relogin` to force relogin


...

                   all         25       2699      0.627      0.279      0.329      0.189      0.513      0.253      0.312      0.164



20 epochs completed in 0.100 hours.
Optimizer stripped from blood_vessel_segmentation/train_20_epochs_1024_batch_4/weights/last.pt, 6.8MB
Optimizer stripped from blood_vessel_segmentation/train_20_epochs_1024_batch_4/weights/best.pt, 6.8MB

Validating blood_vessel_segmentation/train_20_epochs_1024_batch_4/weights/best.pt...
Ultralytics YOLOv8.1.3 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLOv8n-seg summary (fused): 195 layers, 3258259 parameters, 0 gradients, 12.0 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:04<00:00,  1.20s/it]


                   all         25       2699      0.628      0.279      0.329       0.19      0.513      0.252      0.311      0.163
Speed: 0.5ms preprocess, 30.1ms inference, 0.0ms loss, 19.0ms postprocess per image
Results saved to blood_vessel_segmentation/train_20_epochs_1024_batch_4


lr/pg0,▂▄▆███▇▇▆▆▅▅▄▄▃▃▂▂▁▁
lr/pg1,▂▄▆███▇▇▆▆▅▅▄▄▃▃▂▂▁▁
lr/pg2,▂▄▆███▇▇▆▆▅▅▄▄▃▃▂▂▁▁
metrics/mAP50(B),▁▁▄▅▆▆▅▇▇▇▇█████████
metrics/mAP50(M),▁▁▃▅▆▇▅▇▇▇▇▇██▇▇▇▇▇█
metrics/mAP50-95(B),▁▁▄▅▅▆▅▆▇▆▇▇▇▇██████
metrics/mAP50-95(M),▁▁▃▄▆▇▅▇▇█▇▇██▇▇████
metrics/precision(B),▁▁▂▂▆▇▆▇█▇▇█████▇███
metrics/precision(M),▁▁▂▂▇▇▆▇▇█▇▇██▇▇▇▇▇█
metrics/recall(B),▂▁▅▇▆▆▆▇▇▇▇▇████████
metrics/recall(M),▂▁▄▇▆▇▆▇▇█▇▇██▇▇██▇▇


Train image size 640 batching 1 image

In [10]:
model = YOLO('yolov8n-seg.pt')

name = 'train_20_epochs_640_batch_1'

results = model.train(data=yaml_path,
                      project=project,
                      name=name,
                      epochs=20,
                      patience=5,
                      batch=1,
                      imgsz=640,
                      cache=True
                     )

Ultralytics YOLOv8.1.3 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=segment, mode=train, model=yolov8n-seg.pt, data=yolo_dataset.yaml, epochs=20, time=None, patience=5, batch=1, imgsz=640, save=True, save_period=-1, cache=True, device=None, workers=8, project=blood_vessel_segmentation, name=train_20_epochs_640_batch_1, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_

Freezing layer 'model.22.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks with YOLOv8n...
AMP: checks passed ✅


train: Scanning /content/hacking-human-vasculature/dataset/train/labels.cache... 95 images, 5 backgrounds, 0 corrupt: 100%|██████████| 100/100 [00:00<?, ?it/s]
train: Caching images (0.1GB True): 100%|██████████| 100/100 [00:00<00:00, 221.31it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))



val: Scanning /content/hacking-human-vasculature/dataset/val/labels.cache... 25 images, 0 backgrounds, 0 corrupt: 100%|██████████| 25/25 [00:00<?, ?it/s]
val: Caching images (0.0GB True): 100%|██████████| 25/25 [00:00<00:00, 50.31it/s]


...

                   all         25       2699      0.428      0.132      0.144     0.0712       0.35       0.13      0.142     0.0679
Stopping training early as no improvement observed in last 5 epochs. Best results observed at epoch 14, best model saved as best.pt.
To update EarlyStopping(patience=5) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



19 epochs completed in 0.106 hours.
Optimizer stripped from blood_vessel_segmentation/train_20_epochs_640_batch_1/weights/last.pt, 6.8MB
Optimizer stripped from blood_vessel_segmentation/train_20_epochs_640_batch_1/weights/best.pt, 6.8MB

Validating blood_vessel_segmentation/train_20_epochs_640_batch_1/weights/best.pt...
Ultralytics YOLOv8.1.3 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLOv8n-seg summary (fused): 195 layers, 3258259 parameters, 0 gradients, 12.0 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 13/13 [00:08<00:00,  1.62it/s]


                   all         25       2699      0.397      0.127      0.133     0.0649      0.377      0.147       0.16     0.0745
Speed: 0.9ms preprocess, 95.8ms inference, 0.0ms loss, 28.4ms postprocess per image
Results saved to blood_vessel_segmentation/train_20_epochs_640_batch_1


lr/pg0,▃▅███▇▇▆▆▅▅▄▄▃▃▂▂▁▁
lr/pg1,▃▅███▇▇▆▆▅▅▄▄▃▃▂▂▁▁
lr/pg2,▃▅███▇▇▆▆▅▅▄▄▃▃▂▂▁▁
metrics/mAP50(B),▁▅▂▅▆▆▆▆▇█▆████████
metrics/mAP50(M),▁▄▂▃▆▄▆▆▆▆▆▇▇█▇▆▇▇█
metrics/mAP50-95(B),▁▅▃▅▆▅▆▇▆▇▆▇▇██████
metrics/mAP50-95(M),▁▄▂▂▆▄▆▆▆▆▅▇▇█▇▆▆▇█
metrics/precision(B),▁▅▁█▇█▅▆▇▇▆██▆▇██▇▆
metrics/precision(M),▁▄▁█▆▅▄▅▆▅▅▆▆▅▅▆▆▅▅
metrics/recall(B),▁▅▄▃▆▅▇▇▇█▆▆▇██▇▇██
metrics/recall(M),▂▄▂▁▅▃▅▅▆▅▄▅▅█▇▅▅▆█


Train bigger image size and bigger batch


In [12]:
model = YOLO('yolov8n-seg.pt')

name = 'train_20_epochs_1920_batch_9'

results = model.train(data=yaml_path,
                      project=project,
                      name=name,
                      epochs=20,
                      patience=5,
                      batch=9,
                      imgsz=1920,
                      cache=True
                     )

Ultralytics YOLOv8.1.3 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=segment, mode=train, model=yolov8n-seg.pt, data=yolo_dataset.yaml, epochs=20, time=None, patience=5, batch=9, imgsz=1920, save=True, save_period=-1, cache=True, device=None, workers=8, project=blood_vessel_segmentation, name=train_20_epochs_1920_batch_1, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, sho

Freezing layer 'model.22.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks with YOLOv8n...
AMP: checks passed ✅


train: Scanning /content/hacking-human-vasculature/dataset/train/labels.cache... 95 images, 5 backgrounds, 0 corrupt: 100%|██████████| 100/100 [00:00<?, ?it/s]
train: Caching images (0.7GB True): 100%|██████████| 100/100 [00:01<00:00, 72.58it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))



val: Scanning /content/hacking-human-vasculature/dataset/val/labels.cache... 25 images, 0 backgrounds, 0 corrupt: 100%|██████████| 25/25 [00:00<?, ?it/s]
val: Caching images (0.2GB True): 100%|██████████| 25/25 [00:01<00:00, 19.43it/s]


Plotting labels to blood_vessel_segmentation/train_20_epochs_1920_batch_1/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 66 weight(decay=0.0), 77 weight(decay=0.0004921875), 76 bias(decay=0.0)
Image sizes 1920 train, 1920 val
Using 2 dataloader workers
Logging results to blood_vessel_segmentation/train_20_epochs_1920_batch_1
Starting training for 20 epochs...

      Epoch    GPU_mem   box_loss   seg_loss   cls_loss   dfl_loss  Instances       Size


  0%|          | 0/12 [00:01<?, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 256.00 MiB. GPU 0 has a total capacty of 14.75 GiB of which 187.06 MiB is free. Process 5740 has 14.56 GiB memory in use. Of the allocated memory 13.67 GiB is allocated by PyTorch, and 643.89 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [9]:
model = YOLO('yolov8n-seg.pt')

name = 'train_20_epochs_1600_batch_9'

results = model.train(data=yaml_path,
                      project=project,
                      name=name,
                      epochs=20,
                      patience=5,
                      batch=9,
                      imgsz=1600,
                      cache=True
                     )

100%|██████████| 6.73M/6.73M [00:00<00:00, 77.6MB/s]


Ultralytics YOLOv8.1.3 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=segment, mode=train, model=yolov8n-seg.pt, data=yolo_dataset.yaml, epochs=20, time=None, patience=5, batch=9, imgsz=1600, save=True, save_period=-1, cache=True, device=None, workers=8, project=blood_vessel_segmentation, name=train_20_epochs_1600_batch_9, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, sho

100%|██████████| 755k/755k [00:00<00:00, 15.5MB/s]


Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

wandb: Currently logged in as: aaalex-lit. Use `wandb login --relogin` to force relogin


...

                   all         25       2699      0.735      0.406      0.477      0.282        0.6      0.365      0.443      0.233



20 epochs completed in 0.155 hours.
Optimizer stripped from blood_vessel_segmentation/train_20_epochs_1600_batch_9/weights/last.pt, 7.0MB
Optimizer stripped from blood_vessel_segmentation/train_20_epochs_1600_batch_9/weights/best.pt, 7.0MB

Validating blood_vessel_segmentation/train_20_epochs_1600_batch_9/weights/best.pt...
Ultralytics YOLOv8.1.3 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLOv8n-seg summary (fused): 195 layers, 3258259 parameters, 0 gradients, 12.0 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:02<00:00,  1.30s/it]


                   all         25       2699      0.709      0.414      0.476      0.282      0.611      0.374      0.454      0.251
Speed: 0.9ms preprocess, 11.7ms inference, 0.0ms loss, 32.1ms postprocess per image
Results saved to blood_vessel_segmentation/train_20_epochs_1600_batch_9


lr/pg0,▁▂▄▅▆▇▇███▇▆▆▅▄▄▃▂▂▁
lr/pg1,▁▂▄▅▆▇▇███▇▆▆▅▄▄▃▂▂▁
lr/pg2,▁▂▄▅▆▇▇███▇▆▆▅▄▄▃▂▂▁
metrics/mAP50(B),▁▂▁▁▂▃▄▅▆▇▆▇▇███████
metrics/mAP50(M),▁▂▁▁▂▃▄▅▅█▅▆▇▇▇▇▇███
metrics/mAP50-95(B),▁▂▁▁▂▃▄▅▆▆▆▇▇█▇▇████
metrics/mAP50-95(M),▁▂▂▁▂▃▄▄▅▇▅▅▇▇▆▆▆███
metrics/precision(B),▁▁▁▁▁▂▂▇██▇▇█▇▇▇▇█▇▇
metrics/precision(M),▁▁▁▁▁▂▂██▇▆▇▇▇▇▇▇▇▇▇
metrics/recall(B),▁▂▂▂▃▄▆▅▅▇▆▆▇███████
metrics/recall(M),▁▂▂▂▃▄▆▄▅█▅▆▇▇▇▇▇███


For the sake of speed for hyperparameter tuning we'll use   
size 1024  
batch 4